## Compare regions within the same animal

This script allows you to create point clouds for the comparison of projection patterns to four regions of interest within the same animal. The topographical projection pattern depends on where the tracer was injected. We selected 5 cases from the wild type dataset ([DOI: 10.25493/QT31-PJS](https://search.kg.ebrains.eu/instances/4660e79b-a731-40ac-905e-46d0d11c0dd5)). The resulting output file can be used for visualization in [Meshview](https://meshview.apps.hbp.eu/?atlas=ABA_Mouse_CCFv3_2017_25um). 

| Case Number | Injection site |
|--|--|
|100141219 | VISp   |
|100149109 | AUDp   |   
|100140949 | RSPv   |   
|141602484 | MOs    |
|100149969 | SSpm   |

In [1]:
# Import packages
import json
import os
from pathlib import Path


## Create output folder if it does not exist yet

In [2]:
# Data folder
data_folder = os.path.join("..", "data")

# Check if output folder exist, if not create the folder
output_folder = os.path.join("..", "output")
if not os.path.exists(output_folder):
   os.makedirs(output_folder)
   print("The new directory is created!")

## Define the cases and load the file names
Ensure that the experiment files downloaded from EBRAINS are in the data folder

In [3]:
cases = [
    {"name": "100141219_VISp", "color": [0, 0, 0]},
    {"name": "100149109_AUDp", "color": [255, 0, 0]},
    {"name": "100140949_RSPv", "color": [0, 255, 0]},
    {"name": "141602484_MOs", "color": [0, 0, 255]},
    {"name": "100149969_SSpm", "color": [127, 127, 127]}
]

fnames = sorted(fname for case in cases for fname in Path(data_folder).rglob(f"{case['name']}.json"))
fnames = list(map(str, fnames))


## Create the JSON files

In [4]:
region_mapping = {
    "Caudoputamen": "caudo",
    "Thalamus": "thalamus",
    "PontineNuclei": "pons",
    "SuperiorColliculus": "sc"
}

points_dict = {region: [] for region in region_mapping.values()}

for fname, case in zip(fnames, cases):
    exp_name = os.path.splitext(os.path.basename(fname))[0]
    with open(fname, 'r') as f:
        points = json.load(f)

        for d in points:
            region = d['name']
            d["name"] = exp_name
            d["r"], d["g"], d["b"] = map(int, case['color'])

            if region in region_mapping:
                points_dict[region_mapping[region]].append(d)

points_list = [points_dict]

def chunked_json_dump(data, file, chunk_size=1024):
    for i in range(0, len(data), chunk_size):
        file.write(json.dumps(data[i:i+chunk_size], ensure_ascii=False))

# Saving the JSON file
for region, region_key in region_mapping.items():
    out_file = os.path.join(output_folder, f"compare_{region}.json")
    print(f"Saving file {out_file}")
    with open(out_file, 'w', encoding='utf-8') as fi:
        chunked_json_dump(points_dict[region_key], fi)

print("Done!")


Saving file ..\output\compare_Caudoputamen_.json
Saving file ..\output\compare_Thalamus_.json
Saving file ..\output\compare_PontineNuclei_.json
Saving file ..\output\compare_SuperiorColliculus_.json
Done!
